In [1]:
from polygon import RESTClient
from typing import cast
from urllib3 import HTTPResponse
from config import POLYGON_API_KEY
import json
from datetime import datetime
import time
import pandas as pd

In [2]:
# client = RESTClient(POLYGON_API_KEY)

# aggs = cast(
#     HTTPResponse,
#     client.get_aggs(
#         "JPM",
#         1,
#         "day",
#         "2022-09-14",
#         "2022-09-15",
#         raw=True,
#     ),
# )

In [3]:
time.mktime(datetime(2022, 9, 15, 11, 0).timetuple())*1e9

1.663254e+18

In [4]:
client = RESTClient(POLYGON_API_KEY)

quotes_aggs = cast(
    HTTPResponse,
    client.list_quotes(
        ticker="AAPL",
#         timestamp_gte=datetime(2022, 9, 15, 8, 0),
#         timestamp_lte=datetime(2022, 9, 16, 8, 1),
        timestamp_gte=datetime(2021, 4, 19, 11, 50),
        timestamp_lte=datetime(2021, 4, 19, 11, 51),
        limit=50000,
        order="asc",
        raw=True,
    ),
)

trades_aggs = cast(
    HTTPResponse,
    client.list_trades(
        ticker="AAPL",
#         timestamp_gte=datetime(2022, 9, 15, 8, 0),
#         timestamp_lte=datetime(2022, 9, 16, 8, 1),
        timestamp_gte=datetime(2021, 4, 19, 11, 50),
        timestamp_lte=datetime(2021, 4, 19, 11, 51),
        limit=50000,
        order="asc",
        raw=True,
    ),
)

In [5]:
print(quotes_aggs.geturl())
print(quotes_aggs.status)
print(trades_aggs.geturl())
print(trades_aggs.status)
quotes_aggs_res = json.loads(quotes_aggs.data.decode("utf-8"))['results']
trades_aggs_res = json.loads(trades_aggs.data.decode("utf-8"))['results']
print(f"{len(quotes_aggs_res)} quotes")
print(f"{len(trades_aggs_res)} trades")

https://api.polygon.io/v3/quotes/AAPL
200
https://api.polygon.io/v3/trades/AAPL
200
4070 quotes
1150 trades


print(json.dumps(quotes_aggs_res[:2], indent=4))
print(json.dumps(quotes_aggs_res[-2:], indent=4))

print(json.dumps(trades_aggs_res[:2], indent=4))
print(json.dumps(trades_aggs_res[-2:], indent=4))

In [6]:
# https://stackoverflow.com/questions/15649942/how-to-convert-epoch-time-with-nanoseconds-to-human-readable

def nanos2dt(row, ts_col):
    dt = datetime.fromtimestamp(row[ts_col] / 1e9)
    return '{}.{:09.0f}'.format(dt.strftime('%Y-%m-%dT%H:%M:%S'), row[ts_col] % 1e9)

In [7]:
def list_col_len_gt1(row, list_col):
    return(len(row[list_col]) > 1)

In [8]:
def same_ts_dif_prices(df, ts_col, prices_col):
    df = df[[ts_col, prices_col]]
    df = df.groupby(ts_col)[prices_col].apply(set).reset_index()
    return(df[df.apply(lambda row: list_col_len_gt1(row, prices_col), axis=1)])

In [9]:
quotes_df = pd.json_normalize(quotes_aggs_res)

In [10]:
# pt_dt - participant datetime
quotes_df['pt_dt'] = quotes_df.apply(lambda row: nanos2dt(row, 'participant_timestamp'), axis=1)
quotes_df['st_dt'] = quotes_df.apply(lambda row: nanos2dt(row, 'sip_timestamp'), axis=1)

In [11]:
same_ts_dif_prices(quotes_df, 'pt_dt', 'ask_price')

,pt_dt,ask_price
1366,2021-04-19T11:50:19.211407872,"{134.32, 134.31}"
2865,2021-04-19T11:50:41.727022080,"{134.3, 134.29}"
3648,2021-04-19T11:50:55.730852096,"{134.3, 134.31}"


In [12]:
same_ts_dif_prices(quotes_df, 'pt_dt', 'bid_price')

,pt_dt,bid_price


In [13]:
trades_df = pd.json_normalize(trades_aggs_res)

In [14]:
# pt_dt - participant datetime
trades_df['pt_dt'] = trades_df.apply(lambda row: nanos2dt(row, 'participant_timestamp'), axis=1)
trades_df['st_dt'] = trades_df.apply(lambda row: nanos2dt(row, 'sip_timestamp'), axis=1)

In [15]:
same_ts_dif_prices(trades_df, 'pt_dt', 'price')

,pt_dt,price
75,2021-04-19T11:50:04.136000000,"{134.3099, 134.3}"
343,2021-04-19T11:50:21.768999936,"{134.3088, 134.3004}"
365,2021-04-19T11:50:22.728999936,"{134.31, 134.3099}"
466,2021-04-19T11:50:29.479000064,"{134.295, 134.2987}"
518,2021-04-19T11:50:31.791000064,"{134.29, 134.296}"
529,2021-04-19T11:50:32.630641920,"{134.29, 134.295}"
581,2021-04-19T11:50:36.980999936,"{134.2999, 134.2982}"
590,2021-04-19T11:50:38.387000064,"{134.3, 134.2999}"
642,2021-04-19T11:50:43.782000128,"{134.298, 134.3}"
645,2021-04-19T11:50:43.782605312,"{134.3, 134.295}"


same_ts_dif_prices(quotes_df, 'st_dt', 'ask_price')

same_ts_dif_prices(quotes_df, 'st_dt', 'bid_price')

same_ts_dif_prices(trades_df, 'st_dt', 'price')

In [16]:
quotes_df['st_minus_pt'] = quotes_df.apply(lambda row: ((row.sip_timestamp - row.participant_timestamp)/1000000000), axis=1)
trades_df['st_minus_pt'] = trades_df.apply(lambda row: ((row.sip_timestamp - row.participant_timestamp)/1000000000), axis=1)

In [17]:
quotes_df

,ask_exchange,ask_price,ask_size,bid_exchange,bid_price,bid_size,conditions,indicators,participant_timestamp,sequence_number,sip_timestamp,tape,pt_dt,st_dt,st_minus_pt
0,12,134.30,1,11,134.30,1,[1],[1],1618847400021580667,43754568,1618847400021593540,3,2021-04-19T11:50:00.021580544,2021-04-19T11:50:00.021593600,0.000013
1,12,134.30,2,11,134.30,1,[1],[1],1618847400021636221,43754572,1618847400021652525,3,2021-04-19T11:50:00.021636096,2021-04-19T11:50:00.021652480,0.000016
2,12,134.30,2,12,134.29,14,[1],[1],1618847400021332992,43754575,1618847400021699033,3,2021-04-19T11:50:00.021332992,2021-04-19T11:50:00.021699072,0.000366
3,10,134.30,3,12,134.29,13,[1],[1],1618847400021405952,43754589,1618847400021770795,3,2021-04-19T11:50:00.021405952,2021-04-19T11:50:00.021770752,0.000365
4,11,134.30,4,12,134.29,13,[1],[1],1618847400021433088,43754592,1618847400021795440,3,2021-04-19T11:50:00.021433088,2021-04-19T11:50:00.021795328,0.000362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4065,12,134.33,3,19,134.32,12,[1],[1],1618847459839767292,44007812,1618847459839780679,3,2021-04-19T11:50:59.839767296,2021-04-19T11:50:59.839780608,0.000013
4066,12,134.33,3,8,134.32,11,[1],[1],1618847459844618000,44007823,1618847459844799440,3,2021-04-19T11:50:59.844617984,2021-04-19T11:50:59.844799488,0.000181
4067,12,134.33,3,12,134.32,8,[1],[1],1618847459844618000,44007824,1618847459844800238,3,2021-04-19T11:50:59.844617984,2021-04-19T11:50:59.844800256,0.000182
4068,12,134.33,3,19,134.32,7,[1],[1],1618847459861214826,44007900,1618847459861227039,3,2021-04-19T11:50:59.861214720,2021-04-19T11:50:59.861227008,0.000012


In [18]:
trades_df

,conditions,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp,pt_dt,st_dt,st_minus_pt
0,"[10, 2, 41]",4,92299,1618847400000000000,134.4259,2392599,1618847400012149620,300,3,12.0,1.618847e+18,2021-04-19T11:50:00.000000000,2021-04-19T11:50:00.012149504,0.012150
1,NaN,11,30295,1618847400021332992,134.3000,2392603,1618847400021700740,100,3,NaN,NaN,2021-04-19T11:50:00.021332992,2021-04-19T11:50:00.021700864,0.000368
2,NaN,4,92300,1618847400053000000,134.2919,2392605,1618847400054706500,100,3,12.0,1.618847e+18,2021-04-19T11:50:00.052999936,2021-04-19T11:50:00.054706432,0.001706
3,[37],4,92301,1618847400155000000,134.2992,2392629,1618847400157785691,1,3,12.0,1.618847e+18,2021-04-19T11:50:00.155000064,2021-04-19T11:50:00.157785600,0.002786
4,NaN,4,92302,1618847400248000000,134.2914,2392634,1618847400249824476,100,3,12.0,1.618847e+18,2021-04-19T11:50:00.248000000,2021-04-19T11:50:00.249824512,0.001824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,NaN,4,30838,1618847459837000000,134.3300,2403115,1618847459840460834,1000,3,10.0,1.618847e+18,2021-04-19T11:50:59.836999936,2021-04-19T11:50:59.840460800,0.003461
1146,[37],4,30839,1618847459846000000,134.3210,2403116,1618847459850232719,19,3,10.0,1.618847e+18,2021-04-19T11:50:59.846000128,2021-04-19T11:50:59.850232832,0.004233
1147,[37],4,30840,1618847459847000000,134.3210,2403117,1618847459851155722,38,3,10.0,1.618847e+18,2021-04-19T11:50:59.847000064,2021-04-19T11:50:59.851155712,0.004156
1148,[37],4,92666,1618847459935000000,134.3211,2403122,1618847459936833389,1,3,12.0,1.618847e+18,2021-04-19T11:50:59.935000064,2021-04-19T11:50:59.936833280,0.001833


In [19]:
same_pt_ts_i = trades_df.participant_timestamp.isin(quotes_df.participant_timestamp)
trades_df[same_pt_ts_i]

,conditions,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp,pt_dt,st_dt,st_minus_pt
1,NaN,11,30295,1618847400021332992,134.30,2392603,1618847400021700740,100,3,NaN,NaN,2021-04-19T11:50:00.021332992,2021-04-19T11:50:00.021700864,0.000368
7,"[14, 37, 41]",12,61007,1618847400366924254,134.29,2392667,1618847400366940184,5,3,NaN,NaN,2021-04-19T11:50:00.366924288,2021-04-19T11:50:00.366940160,0.000016
8,[37],12,61008,1618847400367240183,134.29,2392668,1618847400367256693,95,3,NaN,NaN,2021-04-19T11:50:00.367240192,2021-04-19T11:50:00.367256576,0.000017
9,NaN,12,61009,1618847400367240183,134.29,2392669,1618847400367258537,400,3,NaN,NaN,2021-04-19T11:50:00.367240192,2021-04-19T11:50:00.367258624,0.000018
10,[37],12,61010,1618847400367240183,134.29,2392670,1618847400367261047,82,3,NaN,NaN,2021-04-19T11:50:00.367240192,2021-04-19T11:50:00.367260928,0.000021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,[37],12,61235,1618847455960346725,134.32,2402583,1618847455960362571,39,3,NaN,NaN,2021-04-19T11:50:55.960346624,2021-04-19T11:50:55.960362496,0.000016
1097,"[14, 37, 41]",11,30378,1618847456496822272,134.32,2402671,1618847456497187446,99,3,NaN,NaN,2021-04-19T11:50:56.496822272,2021-04-19T11:50:56.497187328,0.000365
1107,NaN,12,61237,1618847457902659802,134.32,2402839,1618847457902674560,100,3,NaN,NaN,2021-04-19T11:50:57.902659840,2021-04-19T11:50:57.902674432,0.000015
1142,"[14, 37, 41]",12,61241,1618847459839369771,134.33,2403112,1618847459839385992,19,3,NaN,NaN,2021-04-19T11:50:59.839369728,2021-04-19T11:50:59.839386112,0.000016


In [20]:
same_pt_ts = set(trades_df.participant_timestamp[same_pt_ts_i])
same_pt_ts

{1618847400021332992,
 1618847400366924254,
 1618847400367160832,
 1618847400367240183,
 1618847400367681000,
 1618847401383203000,
 1618847401403692000,
 1618847401868305000,
 1618847402981603072,
 1618847403079133846,
 1618847403079192576,
 1618847404135511000,
 1618847404135778000,
 1618847404135861358,
 1618847404412016000,
 1618847404412213248,
 1618847409228301551,
 1618847410580683000,
 1618847410580835000,
 1618847410580882176,
 1618847411569119000,
 1618847411569185000,
 1618847411569486565,
 1618847412452824000,
 1618847412751165184,
 1618847414451787000,
 1618847414451902464,
 1618847415467813000,
 1618847415467899904,
 1618847417143051000,
 1618847417143342848,
 1618847417144407296,
 1618847417706209569,
 1618847417706731212,
 1618847417706811000,
 1618847417706920000,
 1618847417707151872,
 1618847418766211999,
 1618847418766419611,
 1618847418766592000,
 1618847418766602000,
 1618847418766883840,
 1618847419031802052,
 1618847419031830272,
 1618847419031865000,
 161884741

In [21]:
quotes_df[quotes_df.participant_timestamp.isin(same_pt_ts)]


,ask_exchange,ask_price,ask_size,bid_exchange,bid_price,bid_size,conditions,indicators,participant_timestamp,sequence_number,sip_timestamp,tape,pt_dt,st_dt,st_minus_pt
2,12,134.30,2,12,134.29,14,[1],[1],1618847400021332992,43754575,1618847400021699033,3,2021-04-19T11:50:00.021332992,2021-04-19T11:50:00.021699072,0.000366
22,15,134.30,4,12,134.29,5,[1],[1],1618847400366924254,43755780,1618847400366938140,3,2021-04-19T11:50:00.366924288,2021-04-19T11:50:00.366938112,0.000014
23,15,134.30,4,10,134.29,4,[1],[1],1618847400367240183,43755783,1618847400367257107,3,2021-04-19T11:50:00.367240192,2021-04-19T11:50:00.367257088,0.000017
37,17,134.29,13,19,134.29,2,[1],[1],1618847400367160832,43755837,1618847400367528389,3,2021-04-19T11:50:00.367160832,2021-04-19T11:50:00.367528448,0.000368
42,12,134.29,13,8,134.28,8,[1],[1],1618847400367681000,43755881,1618847400367882928,3,2021-04-19T11:50:00.367681024,2021-04-19T11:50:00.367883008,0.000202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3802,12,134.32,2,12,134.31,8,[1],[1],1618847455960145000,43991982,1618847455960346161,3,2021-04-19T11:50:55.960144896,2021-04-19T11:50:55.960346112,0.000201
3803,19,134.32,1,12,134.31,8,[1],[1],1618847455960346725,43991983,1618847455960360385,3,2021-04-19T11:50:55.960346624,2021-04-19T11:50:55.960360448,0.000014
3835,11,134.32,1,12,134.31,15,[1],[1],1618847456496822272,43993613,1618847456497187074,3,2021-04-19T11:50:56.496822272,2021-04-19T11:50:56.497187072,0.000365
3873,8,134.33,7,12,134.31,7,[1],[1],1618847457902659802,43999133,1618847457902675052,3,2021-04-19T11:50:57.902659840,2021-04-19T11:50:57.902674944,0.000015


In [22]:
trades_df_isin_same_pt_ts = trades_df[trades_df.participant_timestamp.isin(same_pt_ts)]
print(sum(trades_df_isin_same_pt_ts['size']))
trades_df_isin_same_pt_ts

42363


,conditions,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp,pt_dt,st_dt,st_minus_pt
1,NaN,11,30295,1618847400021332992,134.30,2392603,1618847400021700740,100,3,NaN,NaN,2021-04-19T11:50:00.021332992,2021-04-19T11:50:00.021700864,0.000368
7,"[14, 37, 41]",12,61007,1618847400366924254,134.29,2392667,1618847400366940184,5,3,NaN,NaN,2021-04-19T11:50:00.366924288,2021-04-19T11:50:00.366940160,0.000016
8,[37],12,61008,1618847400367240183,134.29,2392668,1618847400367256693,95,3,NaN,NaN,2021-04-19T11:50:00.367240192,2021-04-19T11:50:00.367256576,0.000017
9,NaN,12,61009,1618847400367240183,134.29,2392669,1618847400367258537,400,3,NaN,NaN,2021-04-19T11:50:00.367240192,2021-04-19T11:50:00.367258624,0.000018
10,[37],12,61010,1618847400367240183,134.29,2392670,1618847400367261047,82,3,NaN,NaN,2021-04-19T11:50:00.367240192,2021-04-19T11:50:00.367260928,0.000021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,[37],12,61235,1618847455960346725,134.32,2402583,1618847455960362571,39,3,NaN,NaN,2021-04-19T11:50:55.960346624,2021-04-19T11:50:55.960362496,0.000016
1097,"[14, 37, 41]",11,30378,1618847456496822272,134.32,2402671,1618847456497187446,99,3,NaN,NaN,2021-04-19T11:50:56.496822272,2021-04-19T11:50:56.497187328,0.000365
1107,NaN,12,61237,1618847457902659802,134.32,2402839,1618847457902674560,100,3,NaN,NaN,2021-04-19T11:50:57.902659840,2021-04-19T11:50:57.902674432,0.000015
1142,"[14, 37, 41]",12,61241,1618847459839369771,134.33,2403112,1618847459839385992,19,3,NaN,NaN,2021-04-19T11:50:59.839369728,2021-04-19T11:50:59.839386112,0.000016


In [23]:
trades_df_NOTisin_same_pt_ts = trades_df[~trades_df.participant_timestamp.isin(same_pt_ts)]
print(sum(trades_df_NOTisin_same_pt_ts['size']))
trades_df_NOTisin_same_pt_ts

123113


,conditions,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp,pt_dt,st_dt,st_minus_pt
0,"[10, 2, 41]",4,92299,1618847400000000000,134.4259,2392599,1618847400012149620,300,3,12.0,1.618847e+18,2021-04-19T11:50:00.000000000,2021-04-19T11:50:00.012149504,0.012150
2,NaN,4,92300,1618847400053000000,134.2919,2392605,1618847400054706500,100,3,12.0,1.618847e+18,2021-04-19T11:50:00.052999936,2021-04-19T11:50:00.054706432,0.001706
3,[37],4,92301,1618847400155000000,134.2992,2392629,1618847400157785691,1,3,12.0,1.618847e+18,2021-04-19T11:50:00.155000064,2021-04-19T11:50:00.157785600,0.002786
4,NaN,4,92302,1618847400248000000,134.2914,2392634,1618847400249824476,100,3,12.0,1.618847e+18,2021-04-19T11:50:00.248000000,2021-04-19T11:50:00.249824512,0.001824
5,NaN,4,92303,1618847400316000000,134.2950,2392660,1618847400317282934,100,3,12.0,1.618847e+18,2021-04-19T11:50:00.316000000,2021-04-19T11:50:00.317282816,0.001283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,NaN,4,30838,1618847459837000000,134.3300,2403115,1618847459840460834,1000,3,10.0,1.618847e+18,2021-04-19T11:50:59.836999936,2021-04-19T11:50:59.840460800,0.003461
1146,[37],4,30839,1618847459846000000,134.3210,2403116,1618847459850232719,19,3,10.0,1.618847e+18,2021-04-19T11:50:59.846000128,2021-04-19T11:50:59.850232832,0.004233
1147,[37],4,30840,1618847459847000000,134.3210,2403117,1618847459851155722,38,3,10.0,1.618847e+18,2021-04-19T11:50:59.847000064,2021-04-19T11:50:59.851155712,0.004156
1148,[37],4,92666,1618847459935000000,134.3211,2403122,1618847459936833389,1,3,12.0,1.618847e+18,2021-04-19T11:50:59.935000064,2021-04-19T11:50:59.936833280,0.001833


In [31]:
quotes_df.to_csv('./resources/quotes_df.csv')
trades_df.to_csv('./resources/trades_df.csv')

In [25]:
quotes_df[(quotes_df.participant_timestamp == 1663254000159580520) |
         (quotes_df.participant_timestamp == 1663254396341244000)]

,ask_exchange,ask_price,ask_size,bid_exchange,bid_price,bid_size,conditions,indicators,participant_timestamp,sequence_number,sip_timestamp,tape,pt_dt,st_dt,st_minus_pt


In [26]:
# trades_df[(trades_df.participant_timestamp == 1663270876736038000) |
#          (trades_df.participant_timestamp == 1663270875763496000)]
trades_df[(trades_df.participant_timestamp == 1663254000159580520) |
         (trades_df.participant_timestamp == 1663254396341244000)]

,conditions,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp,pt_dt,st_dt,st_minus_pt


In [27]:
len(set(quotes_df.participant_timestamp)), len(set(quotes_df.sip_timestamp))

(4046, 4070)

In [28]:
len(set(quotes_df.ask_price)), len(set(quotes_df.bid_price))

(7, 7)

In [29]:
len(set(trades_df.participant_timestamp)), len(set(trades_df.sip_timestamp))

(865, 1150)

In [30]:
a_tuple = ((1,2), 'df')
print(a_tuple)
type(a_tuple)

((1, 2), 'df')


tuple